In [1]:
import re
import os
import csv
import json
import requests
from urllib import parse
from bs4 import BeautifulSoup
from locals.output_data import exportData
from locals.date_scrape import getDate, lastStamp
from locals.automate import By, Keys, EC, Action, wait_for, genBrowser, driverWait

In [12]:
chrome = genBrowser()

In [13]:
chrome.get('https://www.accenture.com/au-en/careers/jobsearch?jk=&sb=1&vw=0&is_rj=0&pg=1')

In [14]:
chrome.refresh()

In [3]:
session = requests.Session()
header = {
    'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}
def get_desc(job_url):
    second_url = job_url
    res = session.get(second_url, headers=header)
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        job_description = soup.select('div.cmp-job-listing-details__description-content')
        job_desc = job_description[0].text.replace('\n', '')
        job_quali = job_description[1].text.replace('\n', '')
        
        job_details = {
            'job_description': job_desc,
            'job_qualification': job_quali
        }

        return job_details

In [ ]:
job_list = []

In [15]:

for page in range(1, 22):

    # if page >= 5:
    #     break
    base_url = f'https://www.accenture.com/au-en/careers/jobsearch?jk=&sb=1&vw=0&is_rj=0&pg={page}'
    chrome.get(base_url)

    while True:
        job_cards = chrome.find_elements(By.CSS_SELECTOR, 'div.wrapper.teaser.dcc-job-card > div.cmp-teaser.card')
        if not len(job_cards):
            wait_for(1)
        else:
            print(f'\ntotal job in this page: {len(job_cards)}\n')
            break

    # data collect
    for job in job_cards:
        job_html = job.get_attribute('outerHTML')
        soup = BeautifulSoup(job_html, 'lxml')
        job_title = soup.select_one('div.cmp-teaser__content div.cmp-teaser__content__details-section h3').string.replace('*', '')
        job_region = soup.select_one('div.cmp-teaser__content div.cmp-teaser__address-location div.cmp-teaser-region').string
        job_city = soup.select_one('div.cmp-teaser__content div.cmp-teaser__address-location div.cmp-teaser-city').string
        job_field = soup.select_one('div.cmp-teaser__content div.cmp-teaser__description span.skill').string
        job_url_tag = soup.select_one('div.cmp-teaser.card > a.cmp-teaser__title-link')
        job_url = job_url_tag['href']
        job_post_text = soup.select_one('div.cmp-teaser__content p.cmp-teaser__job-listing-posted-date').string
        job_post = re.findall(r'\d+', job_post_text)[0]
        
        job_details = get_desc(job_url)
        job_location = f'{job_region}, {job_city}'

        job_dict = {
            'title': job_title,
            'address': job_location,
            'field': job_field,
            'description': job_details['job_description'],
            'qualification': job_details['job_qualification'],
            'url': job_url,
            'posted_ago': job_post
        }

        job_list.append(job_dict)


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9


total job in this page: 9



In [21]:
chrome.quit()

In [23]:
len(job_list)

747

In [9]:
job_list[1]

{'title': 'Financial Instrument Lead - 4871244',
 'address': 'None, Multiple Locations ',
 'field': 'Program, Project & Service Management',
 'description': "Accenture Flex offers you the flexibility of local fixed-duration project-based work powered by Accenture, a leading global professional services company. Accenture is consistently recognized on FORTUNE's 100 Best Companies to Work For and Diversity Inc's Top 50 Companies For Diversity lists.As an Accenture Flex employee, you will apply your skills and experience to help drive business transformation for leading organizations and communities. In addition to delivering innovative solutions for Accenture's clients, you will work with a highly skilled, diverse network of people across Accenture businesses who are using the latest emerging technologies to address today's biggest business challenges.You will receive competitive rewards and access to benefits programs and world-class learning resources. Accenture Flex employees work in 

In [29]:
import re
from datetime import datetime, date

def set_date(day_ago):
    days = int(day_ago)

    if days > 5:
        date_str = '2023-06-10'
    else:
        time_stamp = 86400 * days
        now_date = date.today()
        today_obj = datetime.strptime(str(now_date), '%Y-%m-%d')
        today_stamp = today_obj.timestamp()
        final_date_stamp = today_stamp - time_stamp
        final_date = int(final_date_stamp)
        dt_obj = datetime.fromtimestamp(final_date)
        date_str = dt_obj.strftime('%Y-%m-%d')

    return date_str

In [30]:
y = set_date(9)
print(y)

2023-06-10


In [31]:
job_list[1]

{'title': 'Financial Instrument Lead - 4871244',
 'address': 'None, Multiple Locations ',
 'field': 'Program, Project & Service Management',
 'description': "Accenture Flex offers you the flexibility of local fixed-duration project-based work powered by Accenture, a leading global professional services company. Accenture is consistently recognized on FORTUNE's 100 Best Companies to Work For and Diversity Inc's Top 50 Companies For Diversity lists.As an Accenture Flex employee, you will apply your skills and experience to help drive business transformation for leading organizations and communities. In addition to delivering innovative solutions for Accenture's clients, you will work with a highly skilled, diverse network of people across Accenture businesses who are using the latest emerging technologies to address today's biggest business challenges.You will receive competitive rewards and access to benefits programs and world-class learning resources. Accenture Flex employees work in 

In [32]:
for job in job_list:
    days_ago = job['posted_ago']

    new_date_str = set_date(days_ago)

    job['posted_ago'] = new_date_str

    print(job['posted_ago'])

2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-13
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-12
2023-06-10
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-09
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10
2023-06-10

In [33]:
exportData('accenture_jobs', job_list)

--Data Saved as JSON file!
--Data Saved as CSV file!
